## Part 3 (8 points total)

## 3. Train single shape SDF

The signed distance function (or oriented distance function) of a set Ω in a metric space determines the distance of a given point x from the boundary of Ω, with the sign determined by whether x is in Ω. The function has positive values at points x inside Ω, it decreases in value as x approaches the boundary of Ω where the signed distance function is zero, and it takes negative values outside of Ω.

<div id="banner">    
    <div class="" style="max-width: 30%;max-height: 20%;display: block;  margin-left: auto;
  margin-right: auto;">
        <img  src="img/440px-Signed_distance2.png">
    </div>
</div>

A set (top) and its signed distance function (bottom, in red).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#single shape SDF

import torch
import torch.nn.functional as F
from torch import nn



class SDFNet(nn.Module):
    def __init__(self, inner_dim=512, num_layers=8, dropout_rate=0.3):
        super(SDFNet, self).__init__()
        self.layers = nn.ModuleList()
        
        input_dim = 3
        
        for i in range(num_layers-1):
            self.layers.append(nn.Linear(input_dim, inner_dim))
            input_dim = inner_dim
        self.output = nn.Linear(inner_dim, 1)
        self.dropout = nn.Dropout(dropout_rate)
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        for layer in self.layers:
            x = F.relu(layer(x), inplace=True)
            x = self.dropout(x)
        
        x = self.tanh(self.output(x))
        return x 

In [3]:
from torch.utils.data import Dataset


class SDFItemDataset(Dataset):
    def __init__(self, point_cloud, sdf):
        '''
        point_cloud: xyz numpy array of shape (n_points, 3)
        sdf: sdf values of shape (n_points, 1)
        '''
        self.point_cloud = point_cloud
        self.sdf = sdf

    def __getitem__(self, index):
        return self.point_cloud[index], self.sdf[index]

    def __len__(self):
        return self.point_cloud.shape[0]

In [4]:
def get_weights(labels):
    total = len(labels)
    weight_negative = float((labels>=0).sum())/total
    return [weight_negative if l<0 else 1-weight_negative for l in labels]

In [5]:
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np

import os
os.environ['CUDA_VISIBLE_DEVICES']='0'

from tqdm.notebook import tqdm as tqdm

import sys 
sys.path.append('..')

import warnings
warnings.filterwarnings('ignore')

## Loading single mesh and corresponding SDF values

In [6]:
import trimesh
# load a file by name or from a buffer
mesh1 = trimesh.load_mesh('data/ShapeNet_data/02876657/6d680415396d14d4d7c5502d4a22edf5/mesh.obj')


unable to load materials from: model_normalized.mtl
specified material not loaded!
specified material not loaded!
specified material not loaded!


In [7]:
mesh1.show()

In [9]:
plane_mesh = np.load(os.path.join('data/ShapeNet_data/02876657/6d680415396d14d4d7c5502d4a22edf5/pcloud.npy'))
plane_sdf = np.load(os.path.join('data/ShapeNet_data/02876657/6d680415396d14d4d7c5502d4a22edf5/sdf.npy'))
plane_sdf = np.expand_dims(plane_sdf,-1)

assert plane_sdf.ndim==2 and plane_sdf.shape[1]==1
assert plane_mesh.ndim==2 and plane_mesh.shape[1]==3

In [29]:
batch_size = 16 ** 3
train_steps = 400
val_steps = 200

# random points for validation
val_fraction = 0.35

val_mask = np.zeros((plane_mesh.shape[0]), dtype=np.bool)
val_ind = np.random.choice(range(plane_mesh.shape[0]), int(val_fraction*plane_mesh.shape[0]))
val_mask[val_ind] = 1

train_dataset = SDFItemDataset(plane_mesh[~val_mask], plane_sdf[~val_mask])
val_dataset = SDFItemDataset(plane_mesh[val_mask], plane_sdf[val_mask])

# balanced sampling: 1:1 positive:negative 
weights_train = get_weights(plane_sdf[~val_mask])
weights_val = get_weights(plane_sdf[val_mask])

train_sampler = WeightedRandomSampler(weights_train, batch_size*train_steps)
val_sampler = WeightedRandomSampler(weights_val, batch_size*val_steps)

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=train_sampler, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=val_sampler, num_workers=4)

<b>Task 3a. (2 points)</b>  Implement clamp function according to the paper

In [14]:
# to clamp sdf values for both targets and model outputs
#def clamp(delta, x):
    #######

def mse(outputs, targets):
    return ((outputs - targets) ** 2).sum() 

<b>Task 3b. (2 points)</b>  Implement loss

In [30]:
model = SDFNet().cuda()

####### criterion = 

lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)

In [32]:
class SDFTrainer:
    def __init__(self, model, criterion, optimizer, delta=0.1, checkpoints_dir='checkpoints'):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.delta = delta
        os.makedirs(checkpoints_dir, exist_ok=True)
        self.checkpoints_dir=checkpoints_dir
        
    def fit(self, train_loader, val_loader, epochs, save=False):
        best_val_mse = 1e6
        for epoch in range(epochs):
            self._train(train_loader, epoch)
            val_loss, val_mse = self._validate(val_loader)
            if val_mse<best_val_mse:
                best_val_mse = val_mse
                if save:
                    self.save_weights()  
    def save_weights(self, name='model.pth'):
        torch.save(self.model.state_dict(), os.path.join(self.checkpoints_dir, name))

    def load_weights(self, weights_path):
        self.model.load_state_dict(weights_path)
        
    def _validate(self, loader):
        self.model.eval()
        running_loss = []
        running_mse = []
        
        for inputs, targets in loader:
            inputs = inputs.float().cuda()
            targets = targets.float().cuda()

            with torch.set_grad_enabled(False):
                outputs = self.model(inputs)
                
                ###########
                ## loss =
                
                running_loss.append(loss.item())
                running_mse.append(mse(outputs, targets).detach().cpu().numpy())
        
        mean_loss = np.mean(running_loss)
        mean_mse = np.mean(running_mse)
        
        print(f'val loss: {mean_loss:.5f}, val mse: {mean_mse:.5f}')
        return mean_loss, mean_mse
        
    def _train(self, loader, epoch):
        self.model.train()
        running_loss = []
        running_mse = []
        tq = tqdm(total=len(loader))
        tq.set_description('Epoch {}'.format(epoch))
        
        for inputs, targets in loader:
            inputs = inputs.float().cuda()
            targets = targets.float().cuda()

            self.optimizer.zero_grad()
            
            with torch.set_grad_enabled(True):
                outputs = self.model(inputs)
                
                ###########
                ## loss =

            loss.backward()
            self.optimizer.step()

            running_loss.append(loss.item())
            running_mse.append(mse(outputs, targets).detach().cpu().numpy())

            mean_loss = np.mean(running_loss)
            mean_mse = np.mean(running_mse)
                
            tq.update()
            tq.set_postfix(loss='{:.3f}'.format(mean_loss), mse = '{:.3f}'.format(mean_mse))            


<b>Task 3c. (1 points)</b>  Implement loading SDFTrainer class and fit the sdf

In [ ]:
epochs = 2
delta = 0.2

#########
# sdf_trainer =

# sdf_trainer. 

## Visualization

<b>Task 3d. (1 points)</b>  Implement visualisation of point cloud and original sdf as colors

<b>Task 3e. (2 points)</b>  Implement visualisation of point cloud and computed sdf as colors